# Funciones

En este notebook se desarrollan las funciones utilizadas en el proyecto que seran las funciones de los endopoints de la API

##  Elaboración de la función **userdata**

En el siguiente apartado se ve la elaboración de la función userdata, la cual nos pide un **"User-id"** y nos retorna la cantidad de dinero gastada por el usuario así como el porcentaje de recomendación en base a reviews recommend y la cantidad de items que tiene.

In [29]:
import pandas as pd
import numpy as np
import ast

In [30]:
# Cargo el dataframe de Games
df = pd.read_csv("games1.csv")
df.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,early_access,id,developer,release_year,price
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],False,761140,Kotoshiro,2018,4.99
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",False,643980,Secret Level SRL,2018,0.00
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",False,670290,Poolians.com,2017,0.00
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],False,767400,彼岸领域,2017,0.99
4,Sin dato,Sin dato,Log Challenge,Sin dato,http://store.steampowered.com/app/773570/Log_C...,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",False,773570,Sin dato,sin_dato,2.99


In [31]:
df = df.drop(["tags", "specs", "url", "reviews_url", "early_access","genres"], axis=1)
df

,publisher,app_name,title,id,developer,release_year,price
0,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,2018,4.99
1,"Making Fun, Inc.",Ironbound,Ironbound,643980,Secret Level SRL,2018,0.00
2,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,670290,Poolians.com,2017,0.00
3,彼岸领域,弹炸人2222,弹炸人2222,767400,彼岸领域,2017,0.99
4,Sin dato,Log Challenge,Sin dato,773570,Sin dato,sin_dato,2.99
...,...,...,...,...,...,...,...
32127,Ghost_RUS Games,Colony On Mars,Colony On Mars,773640,"Nikita ""Ghost_RUS""",2018,1.99
32128,Sacada,LOGistICAL: South Africa,LOGistICAL: South Africa,733530,Sacada,2018,4.99
32129,Laush Studio,Russian Roads,Russian Roads,610660,Laush Dmitriy Sergeevich,2018,1.99
32130,SIXNAILS,EXIT 2 - Directions,EXIT 2 - Directions,658870,"xropi,stev3ns",2017,4.99


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     32113 non-null  object 
 1   app_name      32132 non-null  object 
 2   title         32132 non-null  object 
 3   id            32132 non-null  int64  
 4   developer     32132 non-null  object 
 5   release_year  32132 non-null  object 
 6   price         32132 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


In [12]:
# Se quitan las filas que tengan title como  sin dato 
df= df[df['title'] != 'sin dato']

In [13]:
# Se genera un punto de restauración 
df_gamesT = df

In [34]:
# Se carga el dataframe de items
df_items = pd.read_csv("itemsGrande.csv")
df_items.head()

,user_id,items_count,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0
3,76561197970982479,277,40,Deathmatch Classic,0,0
4,76561197970982479,277,50,Half-Life: Opposing Force,0,0


In [35]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   item_id           int64 
 3   item_name         object
 4   playtime_forever  int64 
 5   playtime_2weeks   int64 
dtypes: int64(4), object(2)
memory usage: 235.9+ MB


In [36]:
df_items = df_items.drop(["playtime_2weeks","item_name"], axis=1)

In [37]:
# Se unen los dataframes Games y Items
dfG = pd.merge(df_gamesT,df_items, left_on="id", right_on="item_id")

In [18]:
dfG.head()

,publisher,app_name,title,id,developer,release_year,price,user_id,items_count,item_id,playtime_forever
0,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,UTNerd24,188,282010,5
1,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,I_DID_911_JUST_SAYING,154,282010,0
2,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,76561197962104795,79,282010,0
3,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,r3ap3r78,331,282010,0
4,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,saint556,583,282010,13


In [ ]:
# Se guarda como parquet el unido para mayor eficiencia
dfG.to_parquet("GameItem.parquet")

In [ ]:
# se llama al dataframe reviews
df_rev = pd.read_csv("reviewT.csv")

Se encontraron inneficiencias en el código, asi que se crearan archivos pequeños para cada función.
Del dataframe se requiere  "price, "user_id", "items_count" 

In [52]:
# Se guarda una copia antes de quitar las columnas
dfGTotal = dfG
dfG.head()

,publisher,app_name,title,id,developer,release_year,price,user_id,items_count,item_id,playtime_forever
0,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,UTNerd24,188,282010,5
1,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,I_DID_911_JUST_SAYING,154,282010,0
2,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,76561197962104795,79,282010,0
3,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,r3ap3r78,331,282010,0
4,Stainless Games Ltd,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,saint556,583,282010,13


In [53]:
dfUserData =dfG.drop(["publisher", "app_name", "title", "id", "developer", "release_year","item_id", "playtime_forever"], axis=1)

In [54]:
dfUserData.head()

,price,user_id,items_count
0,9.99,UTNerd24,188
1,9.99,I_DID_911_JUST_SAYING,154
2,9.99,76561197962104795,79
3,9.99,r3ap3r78,331
4,9.99,saint556,583


In [44]:
dfUserData.to_parquet("dfUserdata.parquet")

In [45]:
dfUserData = pd.read_parquet("data/dfUserData.parquet")

Los dataframes siguen siendo enormes por lo que se generan tablas de consulta para cada función.

In [46]:
# Calcular la suma de la cantidad de dinero gastado por cada usuario
dinero_usuario = dfG.groupby('user_id')['price'].sum().reset_index()
dinero_usuario.head()

,user_id,price
0,--000--,402.77
1,--ace--,166.82
2,--ionex--,109.92
3,-2SV-vuLB-Kg,437.49
4,-404PageNotFound-,1514.31


In [62]:
#Ya que tengo un dataframe que contiene la cantidad de items requiero formar una unica fila con usuarios unicos
filas_unicas = dfUserData.drop_duplicates(subset=['user_id'])
# Se quita price que ahora no nos sirve 
filas_unicas=filas_unicas.drop(["price"], axis=1)
filas_unicas.head()

,user_id,items_count
0,UTNerd24,188
1,I_DID_911_JUST_SAYING,154
2,76561197962104795,79
3,r3ap3r78,331
4,saint556,583


In [72]:
# Se pueden unir las dos tablas para dejar una sola 
dfUSERData = pd.merge(dinero_usuario,filas_unicas, on="user_id")
dfUSERData

,user_id,price,items_count
0,--000--,402.77,58
1,--ace--,166.82,44
2,--ionex--,109.92,23
3,-2SV-vuLB-Kg,437.49,68
4,-404PageNotFound-,1514.31,149
...,...,...,...
68707,zzonci,19.98,5
68708,zzoptimuszz,64.98,61
68709,zzydrax,99.94,13
68710,zzyfo,828.51,84


In [73]:
# Se guarda como parquet para disminuir su tamaño 

dfUSERData.to_parquet("dfUSERData.parquet")

In [65]:
dfUSERData = pd.read_parquet("dfUSERData.parquet")
dfUSERData.head()

,user_id,price,items_count
0,--000--,402.77,58
1,--ace--,166.82,44
2,--ionex--,109.92,23
3,-2SV-vuLB-Kg,437.49,68
4,-404PageNotFound-,1514.31,149


La funcion requiere que se lea el df de reviews y para ahorrar espacio para el deployment se piensa hacer un dataframe que consuma poca memoria.

El dataframe nuevo solo requiere de las columnas user_id y recommend

In [ ]:
df_reviews = pd.read_parquet("Data/ReviewsItems.parquet")

In [ ]:
# Se quitan columnas innecesarias y se genera el nuevo dataframe 
dfUserdata2 = df_reviews.drop(["posted", "item_id",	"helpful", "user_url","sentiment_analysis"], axis=1)
dfUserdata2.head()

,recommend,user_id
0,True,76561197970982479
1,True,76561197970982479
2,True,76561197970982479
3,True,js41637
4,True,js41637


In [ ]:
# Se guarda como parquet para tener mayor eficiencia
dfUserdata2 = dfUserdata2.to_parquet("dfUserdata2.parquet")

In [66]:
dfUserdata2 = pd.read_parquet("data/dfUserdata2.parquet")

In [70]:
def calcular_cantidad_gastada(userid):
    
    df=dfUSERData
    df2=df[df["user_id"]== userid]
    cantidad=sum(df2["price"].astype(float))
    
    return cantidad

def cantidad_items(userid):
    
    df=dfUSERData
    df3 = df[df["user_id"] == userid]
    items = df3["items_count"].iloc[0]
    
    return items


def porcentaje_recomendacion(userid):
    
    df = pd.read_parquet("data/dfUserdata2.parquet")
    df4 = df[df["user_id"] == userid]
    falsos=(df4['recommend'] == False).sum()
    verdaderos=(df4['recommend'] == True).sum()
    totales=falsos+verdaderos
    
    if verdaderos > 0:
        recomendacion = (verdaderos/totales)*100
    else:
        recomendacion = 0
    
    return round(recomendacion,2)


 
def userdata(userid):
    userid=userid
    cantidad=round(calcular_cantidad_gastada(userid),2)
    items=cantidad_items(userid)
    recomendacion=porcentaje_recomendacion(userid)
    
    return {"cantidad":cantidad, "items":items, "recomendacion": recomendacion}

    

Se comprueba la funcionalidad del codigo con la siguiente prueba 

In [71]:
# dato de prueba 
userdata("76561198282090798")
#userdata("I_DID_911_JUST_SAYING")


{'cantidad': 54.96, 'items': 5, 'recomendacion': 0}

## Elaboración de la función **countreviews**

En el siguiente bloque se elabora la funcion counreviews que es una función que funciona de la siguiente forma:

Se pide al usuario una fecha en formato string y la función entrega el número de usuarios que realizaron reviews entre las fechas dadas, el porcentaje de recomendación de los mismos en base a reviews
recommend.


In [ ]:
# Se importa el archivo de lectura 
df_reviews = pd.read_csv('reviewsTrabajo.csv')
df_reviews.head()

,posted,item_id,helpful,recommend,user_id,user_url,sentiment_analysis
0,2011-11-05,1250,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
1,2011-07-15,22200,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2
2,2011-04-21,43110,No ratings yet,True,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1
3,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1
4,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,js41637,http://steamcommunity.com/id/js41637,1


Para una mayor optimización se genera un dataframe con lo necesario en formato parquet, se obserba que la funcion solo requiere de las columnas:
posted y user_id

In [ ]:
# Se quitan columnas innecesarias
dfCountreviews = df_reviews.drop(["item_id", "helpful", "recommend", "user_url", "sentiment_analysis"], axis=1)
dfCountreviews.head()

,posted,user_id
0,2011-11-05,76561197970982479
1,2011-07-15,76561197970982479
2,2011-04-21,76561197970982479
3,2014-06-24,js41637
4,2013-09-08,js41637


In [ ]:
# Se guarda en formato parquet para mayor eficiencia
dfCountreviews.to_parquet("dfCountreviews.parquet")

In [21]:
dfCountreviews = pd.read_parquet("data/dfCountreviews.parquet")
dfCountreviews.head()

,posted,user_id
0,2011-11-05,76561197970982479
1,2011-07-15,76561197970982479
2,2011-04-21,76561197970982479
3,2014-06-24,js41637
4,2013-09-08,js41637


In [ ]:
def countreviews(start_date, end_date):
    try:
        # Convertir las fechas en formato YYYY-MM-DD al tipo de dato de fecha
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        
        # Validar que la fecha inicial sea menor que la fecha final
        if start_date >= end_date:
            return "Error: La fecha inicial debe ser menor que la fecha final."
        
        # Filtrar el DataFrame por la condición de fechas y contar usuarios únicos
        filtered_reviews = dfCountreviews[
            (df_reviews['posted'] >= start_date.strftime('%Y-%m-%d')) &
            (df_reviews['posted'] <= end_date.strftime('%Y-%m-%d'))
        ]
        num_users = filtered_reviews['user_id'].nunique()
        
        porcentaje = (num_users / (df_reviews['user_id'].nunique())) * 100
        return f"En el rango de fechas {start_date} a {end_date}:\n{num_users} usuarios diferentes realizaron posteos. El porcentaje de recomendación es {porcentaje} %"

    except Exception as e:
        return f"Ocurrió un error: {e}"



In [ ]:
# Ejemplo de uso
start_date = '2011-04-01'
end_date = '2011-04-30'
result = countreviews(start_date, end_date)

print(result)

En el rango de fechas 2011-04-01 00:00:00 a 2011-04-30 00:00:00:
9 usuarios diferentes realizaron posteos. El porcentaje de recomendación es 0.03535234503888758 %


## En este apartado se realiza la función **genre** 

La funcion **genre** nos pide un genero de videojuegos dado por el usuario y nos devuelve el ranking en el que se encuentra analizado bajo la columna PlayTimeForever.


### Ranking de géneros por tiempo de juego

A diferencia de las funciones anteriores se requiere realizar un ranking lo cual implica la creación de un dataframe que contenga
el ranking de los géneros de videojuegos tomando en cuenta el tiempo de juego de cada uno.

In [ ]:
# Se crea un dataframe y se agrupan los generos con playtime_forever
df_ranking = dfG.groupby('genres')['playtime_forever'].sum().reset_index()
# Se ordena por 'playtime_horas'
df_ranking = df_ranking.sort_values(by='playtime_forever', ascending=False)
# Agregar una columna con la posición del ranking
df_ranking['ranking'] = df_ranking['playtime_forever'].rank(ascending=False).astype(int)
df_ranking

,genres,playtime_forever,ranking
0,Action,3113562606,1
9,Indie,1494622404,2
12,RPG,1041022718,3
1,Adventure,909995120,4
14,Simulation,867646306,5
18,Strategy,659363841,6
8,Free to Play,610752945,7
10,Massively Multiplayer,446594080,8
4,Casual,252232854,9
15,Sin dato,160466609,10


In [ ]:
df_ranking.to_parquet("genre_ranking.parquet")

In [24]:
genre_ranking = pd.read_parquet("data/genre_ranking.parquet")

In [27]:
def genre(genero):
    # Busca el ranking para el género de interés
    rank = genre_ranking[genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [28]:
genero = "RPG"
genre(genero)

{'rank': 3}

## En este apartado se realiza la función **userforgenre**

La función **userforgenre** recibe un genero de videojuegos en formato string por parte del usuario y nos devuelve el top 5 de usuarios con más horas de juego en el género dado, co la URL del usuario y su user_id.

In [ ]:
# Se lee el dataframe de referencia 
dfG= pd.read_csv("3unidos.csv")

In [ ]:
dfG.head()

,genres,app_name,id,price,user_id,playtime_forever,user_url
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010,9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
1,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010,9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
2,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010,9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
3,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010,9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
4,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,282010,9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24


Para optimizar recursos se genera un dataframe nuevo con la información necesaria para el consumo en API, por lo que se borran columnas innecesarias.

El dataframe nuevo solo requiere las columnas de genres, user_url y user_id

In [ ]:
# Se eliminan columnas innecesarias
dfUSeforgenre = dfG.drop(["app_name", "id"], axis=1)
dfUSeforgenre.head()

,genres,price,user_id,playtime_forever,user_url
0,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
1,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
2,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
3,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
4,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24


In [ ]:
# Se guarda en formato parquet para mayor eficiencia
dfUSeforgenre.to_parquet("dfUSeforgenre.parquet")

In [31]:
dfUSeforgenre = pd.read_parquet("data/dfUSeforgenre.parquet")
dfUSeforgenre.head()

,genres,price,user_id,playtime_forever,user_url
0,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
1,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
2,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
3,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24
4,"['Action', 'Indie', 'Racing']",9.99,UTNerd24,5,http://steamcommunity.com/id/UTNerd24


Se trataron de subir los archivos a render y se obtuvo un error asi que se reconvienten los parquets a csv

In [32]:
dfUSeforgenre.to_csv("dfUSeforgenre.csv", index=False)

In [33]:
dfUSeforgenre = pd.read_csv("dfUSeforgenre.csv")

In [35]:
def userforgenre(genero):
   
    data_por_genero = dfUSeforgenre[dfUSeforgenre['genres'].apply(lambda x: genero in x)]
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_forever'].sum().nlargest(5).reset_index()
    
    top_users_dict = {}
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']            
        }
        top_users_dict[index + 1] = user_info
        
    return top_users_dict

In [36]:
# Prueba de uso de la función
genero = "Adventure"
userforgenre(genero)

{1: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 2: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 3: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 4: {'user_id': 'thiefofrosesinlalaland',
  'user_url': 'http://steamcommunity.com/id/thiefofrosesinlalaland'},
 5: {'user_id': '76561198064484479',
  'user_url': 'http://steamcommunity.com/profiles/76561198064484479'}}

## En este apartado se realiza la función **developer**

LA función **developer** recibe un developer en formato string y nos devuelve una cantidad de items y porcentaje de contenido Free por año segun empres desarolladora.


In [ ]:
# Se lee el dataframe de referencia 
dfG= pd.read_parquet("GameItem.parquet")
dfG.head()

,publisher,genres,app_name,title,id,developer,release_year,price,user_id,items_count,item_id,playtime_forever
0,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,UTNerd24,188,282010,5
1,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,I_DID_911_JUST_SAYING,154,282010,0
2,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,76561197962104795,79,282010,0
3,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,r3ap3r78,331,282010,0
4,Stainless Games Ltd,Action,Carmageddon Max Pack,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,9.99,saint556,583,282010,13


Para una mayor optimización se genera un dataframe con lo necesario en formato parquet, se obserba que la funcion solo requiere de las columnas:
publisher, realese_year, item_id y price

In [ ]:
dfDeveloper = dfG.drop(["genres", "app_name", "title", "id", "developer", "user_id", "items_count", "playtime_forever"], axis=1)

In [ ]:
dfDeveloper.head()

,publisher,release_year,price,item_id
0,Stainless Games Ltd,1997,9.99,282010
1,Stainless Games Ltd,1997,9.99,282010
2,Stainless Games Ltd,1997,9.99,282010
3,Stainless Games Ltd,1997,9.99,282010
4,Stainless Games Ltd,1997,9.99,282010


In [ ]:
# Se guarda en formato parquet para mayor eficiencia
dfDeveloper.to_parquet("dfDeveloper.parquet")

In [38]:
dfDeveloper = pd.read_parquet("data/dfDeveloper.parquet")

Se trataron de subir los archivos a render y se obtuvo un error asi que se reconvienten los parquets a csv

In [39]:
dfDeveloper.to_csv("dfDeveloper.csv", index=False)

In [41]:
dfDeveloper = pd.read_csv("dfDeveloper.csv")

C:\Users\SEBASTIAN\AppData\Local\Temp\ipykernel_2220\2337278211.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dfDeveloper = pd.read_csv("dfDeveloper.csv")


In [42]:
def developer(desarrollador):
    # Se filtra el dataframe por desarrollador de interés
    developer = dfDeveloper[dfDeveloper['publisher'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad = developer.groupby('release_year')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    free_content = developer[developer['price'] == 0.0].groupby('release_year')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    percentaje_free = (free_content / cantidad * 100).fillna(0).astype(int)

    result = {
        'cantidad_por_año': cantidad.to_dict(),
        'porcentaje_gratis_por_año': percentaje_free.to_dict()
    }
    
    return result

In [43]:
desarrollador = 'SEGA'
developer(desarrollador)

{'cantidad_por_año': {2004: 3876,
  2006: 28242,
  2008: 3539,
  2009: 5353,
  2010: 26708,
  2011: 11598,
  2012: 13901,
  2013: 7946,
  2014: 211,
  2015: 1409,
  '2000': 940,
  '2002': 666,
  '2006': 1057,
  '2012': 2509,
  '2013': 4242,
  '2014': 16196,
  '2015': 1344,
  '2016': 1599},
 'porcentaje_gratis_por_año': {2004: 0,
  2006: 0,
  2008: 0,
  2009: 0,
  2010: 0,
  2011: 0,
  2012: 0,
  2013: 0,
  2014: 0,
  2015: 0,
  '2000': 0,
  '2002': 0,
  '2006': 0,
  '2012': 0,
  '2013': 0,
  '2014': 0,
  '2015': 0,
  '2016': 95}}

## En este apartado se realiza la función **sentiment_analysis**

La función **sentiment_analysis** recibe un año en formato int y nos devuelve la cantidad de registros de reseñas que se encuentran categorizadas por un analisis de sentimiento.
retornando el numero de cada una por categoria Negative, Positive, Neutral.


In [ ]:
# Se leen los archivos necesarios para su funcionamiento

df_developer = pd.read_csv("SentimentDfFinal.csv")
df_developer.head()

,developer,release_year,Unnamed: 0,user_id,posted,item_id,helpful,recommend,user_url,sentiment_analysis
0,Stainless Games Ltd,1997,9849,UTNerd24,2014-05-06,244210,1 of 1 people (100%) found this review helpful,True,http://steamcommunity.com/id/UTNerd24,1
1,Stainless Games Ltd,1997,9849,UTNerd24,2015-12-02,440,1 of 2 people (50%) found this review helpful,True,http://steamcommunity.com/id/UTNerd24,1
2,Stainless Games Ltd,1997,9849,UTNerd24,2015-06-06,376920,13 of 27 people (48%) found this review helpful,True,http://steamcommunity.com/id/UTNerd24,1
3,Stainless Games Ltd,1997,9849,UTNerd24,2015-09-29,257750,No ratings yet,True,http://steamcommunity.com/id/UTNerd24,1
4,Stainless Games Ltd,1997,9849,UTNerd24,2014-09-08,221910,No ratings yet,True,http://steamcommunity.com/id/UTNerd24,0


Para una mayor optimización se genera un dataframe con lo necesario en formato parquet, se obserba que la funcion solo requiere de las columnas:
developer, realese_year y sentiment_analysis

In [ ]:
dfSentiment = df_developer.drop(["Unnamed: 0", "user_id", "posted", "item_id", "helpful", "recommend", "user_url"], axis=1)

In [ ]:
# Se guarda dfSentiment como tipo parquet para aumentar su eficiencia
dfSentiment.to_parquet("dfSentiment.parquet")

In [45]:
dfSentiment = pd.read_parquet("data/dfSentiment.parquet")

Se trataron de subir los archivos a render y se obtuvo un error asi que se reconvienten los parquets a csv

In [46]:
dfSentiment.to_csv("dfSentiment.csv", index=False)

In [47]:
dfSentiment = pd.read_csv("dfSentiment.csv")

In [48]:
def sentiment_analysis(empresa_desarrolladora: str):
    # Filtrar el DataFrame por la empresa desarrolladora
    data_por_desarrolladora = dfSentiment[dfSentiment['developer'].apply(lambda x: empresa_desarrolladora in x)]
    
    # Agrupar por año de lanzamiento y análisis de sentimiento, contar el número de registros
    analysis_counts = data_por_desarrolladora.groupby(['release_year', 'sentiment_analysis']).size().unstack(fill_value=0)
    
    # Convertir los resultados a un diccionario
    result_dict = {
        'Negative': analysis_counts[0].to_dict(),
        'Neutral': analysis_counts[1].to_dict(),
        'Positive': analysis_counts[2].to_dict()
    }
    
    return result_dict

In [49]:
#Prueba de uso de la función 
sentiment_analysis("SEGA")

{'Negative': {'2011': 594, '2012': 510, '2013': 34, '2014': 263},
 'Neutral': {'2011': 4031, '2012': 3420, '2013': 305, '2014': 1985},
 'Positive': {'2011': 1868, '2012': 1722, '2013': 129, '2014': 887}}